# 악보 학습
- 악보는 시계열이고 음계가 문장보다 코드화가 쉬움
- 음의 코드와 음의 길이로 데이터 구성
- C(도), D(레), E(미), F(파), G(솔), A(라), B(시)
- 4(4분음표), 8(8분음표)

<img src="https://drive.google.com/uc?id=1wS1Inw0sxAzN8AmwCBHTW2nJ78NYvETy">

- 첫 두 마디(G8 E8 E4  F8 D8 D4)에서 4개 음표 입력으로 다음 음표를 예측하려면
  - G8 E8 E4  F8 **D8** : 1 ~ 4번째 음표, 5번째 음표
  - E8 E4  F8 D8 **D4** : 2 ~ 5번째 음표, 6번째 음표
  --> 2개의 샘플은 4개의 입력(feature, 속성)과 1개의 라벨 값(class)으로 구성
  --> 윈도우 크기 : 4
  
<img src="https://drive.google.com/uc?id=1uWsarl3yPVw7091Km7L9NytRq30SrKoB">

# 학습 과정
- 첫 4개의 음표를 입력하면 나머지를 연주할 수 있는 모델을 만드는 것이 목표

- 학습과정
  - 파란색 박스가 입력값이고, 빨간색 박스가 원하는 출력값
  - 1 ~4번째 음표를 데이터로 5번째 음표를 라벨값으로 학습시킴
  - 다음에는 2 ~ 5번째 음표를 데이터로 6번째 음표를 라벨값으로 학습시킴
  - 이후 한 음표씩 넘어가면서 노래 끝까지 학습시킴

<img src="https://drive.google.com/uc?id=1STyQIHBwh7FKJclpn7ueHO0McNtBzVTo">

- 나비야 악보 데이터

In [1]:
# 코드 기준 번호 정의(인코딩)
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}
# 번호 기준 코드 정의(디코딩)
idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}
# 시퀀스 데이터 정의
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8',
       'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4',
       'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8',
       'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8',
       'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8',
       'g8', 'e8', 'e8', 'e4']

- 음계를 플레이하는 플레이어 만들기

In [2]:
# 소리변환 함수
# o : 옥타브, s : 음표
def freq(o, s):
    if s[0] == 'c':     return 524*2**o
    elif s[0] == 'd':   return 587*2**o
    elif s[0] == 'e':   return 659*2**o
    elif s[0] == 'f':   return 698*2**o
    elif s[0] == 'g':   return 784*2**o
    elif s[0] == 'a':   return 880*2**o
    elif s[0] == 'b':   return 988*2**o

In [3]:
from IPython.display import Audio, display
import numpy as np
import time
def single_tone(frequecy, s, sampling_rate=22050, duration=0.1):
    # frequency: 주파수
    # s : 음표 (4분음표, 8분음표)
    # sampling_rate: 초당 샘플링 데이터 수
    # duration: 지속 시간. 단위 초. 디폴트 1초
    t = np.linspace(0, duration, int(sampling_rate))
    y = np.sin(2 * np.pi * frequecy * t)
    time.sleep(s)
    return y
def play_sound(seq) :
  result = []
  for i in range(len(seq)) :
    # 딜레이를 조절하여 박자를 맞춤
    if seq[i][1] == "4" :
        result.append(single_tone(freq(2, seq[i]), 0.4))
    elif seq[i][1] == "8" :
        result.append(single_tone(freq(2, seq[i]), 0.2))
    octave = np.hstack(result)
  display(Audio(octave, rate=44100))

In [4]:
play_sound(seq)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# 악보를 특성과 라벨데이터 형태로 분리하는 함수
# seq 데이터를 window_size(특성 데이터)개수로 분리
def seq2dataset(seq, window_size) :
  dataset = []

  for i in range(len(seq) - window_size) :
    subset = seq[i : (i + window_size + 1)]
    dataset.append([code2idx[item] for item in subset])

  return dataset

In [8]:
dataset = seq2dataset(seq, 4)

dataset

[[11, 9, 2, 10, 8],
 [9, 2, 10, 8, 1],
 [2, 10, 8, 1, 7],
 [10, 8, 1, 7, 8],
 [8, 1, 7, 8, 9],
 [1, 7, 8, 9, 10],
 [7, 8, 9, 10, 11],
 [8, 9, 10, 11, 11],
 [9, 10, 11, 11, 4],
 [10, 11, 11, 4, 11],
 [11, 11, 4, 11, 9],
 [11, 4, 11, 9, 9],
 [4, 11, 9, 9, 9],
 [11, 9, 9, 9, 10],
 [9, 9, 9, 10, 8],
 [9, 9, 10, 8, 1],
 [9, 10, 8, 1, 7],
 [10, 8, 1, 7, 9],
 [8, 1, 7, 9, 11],
 [1, 7, 9, 11, 11],
 [7, 9, 11, 11, 9],
 [9, 11, 11, 9, 9],
 [11, 11, 9, 9, 2],
 [11, 9, 9, 2, 8],
 [9, 9, 2, 8, 8],
 [9, 2, 8, 8, 8],
 [2, 8, 8, 8, 8],
 [8, 8, 8, 8, 8],
 [8, 8, 8, 8, 9],
 [8, 8, 8, 9, 3],
 [8, 8, 9, 3, 9],
 [8, 9, 3, 9, 9],
 [9, 3, 9, 9, 9],
 [3, 9, 9, 9, 9],
 [9, 9, 9, 9, 9],
 [9, 9, 9, 9, 10],
 [9, 9, 9, 10, 4],
 [9, 9, 10, 4, 11],
 [9, 10, 4, 11, 9],
 [10, 4, 11, 9, 2],
 [4, 11, 9, 2, 10],
 [11, 9, 2, 10, 8],
 [9, 2, 10, 8, 1],
 [2, 10, 8, 1, 7],
 [10, 8, 1, 7, 9],
 [8, 1, 7, 9, 11],
 [1, 7, 9, 11, 11],
 [7, 9, 11, 11, 9],
 [9, 11, 11, 9, 9],
 [11, 11, 9, 9, 2]]

In [9]:
dataset= np.array(seq2dataset(seq, 4))
print(dataset.shape)
print(dataset)

(50, 5)
[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]


In [10]:
# 특성과 라벨로 분리
X = dataset[:, :-1]
y = dataset[:, -1]

X.shape, y.shape

((50, 4), (50,))

In [11]:
from tensorflow.keras.utils import to_categorical

# 음표 인덱스를 정규화
max_idx_value = 13

X = X / float(max_idx_value)

y_en = to_categorical(y)

X.shape, y_en.shape

((50, 4), (50, 12))

In [12]:
X

array([[0.84615385, 0.69230769, 0.15384615, 0.76923077],
       [0.69230769, 0.15384615, 0.76923077, 0.61538462],
       [0.15384615, 0.76923077, 0.61538462, 0.07692308],
       [0.76923077, 0.61538462, 0.07692308, 0.53846154],
       [0.61538462, 0.07692308, 0.53846154, 0.61538462],
       [0.07692308, 0.53846154, 0.61538462, 0.69230769],
       [0.53846154, 0.61538462, 0.69230769, 0.76923077],
       [0.61538462, 0.69230769, 0.76923077, 0.84615385],
       [0.69230769, 0.76923077, 0.84615385, 0.84615385],
       [0.76923077, 0.84615385, 0.84615385, 0.30769231],
       [0.84615385, 0.84615385, 0.30769231, 0.84615385],
       [0.84615385, 0.30769231, 0.84615385, 0.69230769],
       [0.30769231, 0.84615385, 0.69230769, 0.69230769],
       [0.84615385, 0.69230769, 0.69230769, 0.69230769],
       [0.69230769, 0.69230769, 0.69230769, 0.76923077],
       [0.69230769, 0.69230769, 0.76923077, 0.61538462],
       [0.69230769, 0.76923077, 0.61538462, 0.07692308],
       [0.76923077, 0.61538462,

# 신경망 설계

In [13]:
X.shape

(50, 4)

- RNN을 사용할 때는 시간 차원을 추가해야 한다.

In [14]:
X_ex = np.reshape(X, (50, 4, 1))

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model1 = Sequential()

# stateful=True : 앞 단의 결과를 다음 단으로 넘긴다.
model1.add(LSTM(128, batch_input_shape=(1, 4, 1), stateful=True))

model1.add(Dense(12, activation="softmax"))

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (1, 128)                  66560     
                                                                 
 dense_1 (Dense)             (1, 12)                   1548      
                                                                 
Total params: 68,108
Trainable params: 68,108
Non-trainable params: 0
_________________________________________________________________


In [20]:
model1.compile(loss="categorical_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

In [21]:
for epoch_idx in range(500) :
  # shuffle=False : 데이터를 섞지 말라고 설정
  model1.fit(X_ex, y_en, epochs=1, batch_size=1, shuffle=False)
  # reset_states() : 상태를 리셋
  model1.reset_states()

50/50 [==============================] - 0s 4ms/step - loss: 9.7578e-05 - accuracy: 1.0000


In [22]:
pred_count = 50  # 최대 예측 개수 정의
# 한 스텝 예측
seq_out = ['g8', 'e8', 'e4', 'f8']
pred_out = model1.predict(X_ex, batch_size=1)
for i in range(pred_count):
    idx = np.argmax(pred_out[i])  # one-hot 인코딩을 인덱스 값으로 변환
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[idx])
    
print("예측데이터 : ", seq_out)
print("실제데이터 : ", seq)

50/50 [==============================] - 1s 2ms/step
예측데이터 :  ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']
실제데이터 :  ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [24]:
seq_out == seq

True

In [23]:
play_sound(seq_out)

Output hidden; open in https://colab.research.google.com to view.

In [26]:
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in
# 코드를 인덱스값으로 변환
seq_in = [code2idx[it] / float(max_idx_value) for it in seq_in]
for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) # 샘플 수, 타입스텝 수, 속성 수
    pred_out = model1.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx / float(max_idx_value))
    seq_in.pop(0)
model1.reset_states()
print("예측데이터 : ", seq_out)
print("실제데이터 : ", seq)

1/1 [==============================] - 0s 21ms/step
예측데이터 :  ['g8', 'e8', 'e4', 'f8', 'e8', 'e8', 'f8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8']
실제데이터 :  ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [27]:
seq_out == seq

False

In [28]:
play_sound(seq_out)

Output hidden; open in https://colab.research.google.com to view.